In [1]:
# !pip install web3
# !pip install python-dotenv
# !pip install requests
# !pip install pandas

## Imports

In [2]:
from web3 import Web3

# For enums
from enum import StrEnum, auto

# To read environment property file
import os
from dotenv import load_dotenv
from pathlib import Path

import requests
import json

# Date calculations
from datetime import datetime, timedelta
import time

# Importing Pandas to create DataFrame
import pandas as pd

## Constants

In [3]:
# Contract types
class Contract_Type(StrEnum):
    SMART = auto()
    NFT = auto()

# Events
class Event(StrEnum):
    BURN = 'Burn'
    COLLECT = 'Collect'
    MINT = 'Mint'
    INCREASE_LIQUIDITY = 'IncreaseLiquidity'
    DECREASE_LIQUIDITY = 'DecreaseLiquidity'
    
# Interested events for the smart contract
CONTRACT_EVENTS = {
    Contract_Type.SMART : [Event.BURN, Event.COLLECT, Event.MINT],
    Contract_Type.NFT: [Event.COLLECT, Event.INCREASE_LIQUIDITY, Event.DECREASE_LIQUIDITY]
}

# Maps the NFT contract event name to corresponding action
NFT_EVENT_TO_ACTION = {Event.DECREASE_LIQUIDITY:'Withdraw', Event.INCREASE_LIQUIDITY:'Deposit', Event.COLLECT:'Collect Fee'}

# How many days back - approximately 1 year
DAYS = 360

# Page size for ether scan search
PAGE_SIZE = 100

# Maximum pages we are going to search; max number of items return = PAGE_SIZE * MAX_PAGES
MAX_PAGES = 5

# Etherscan endpoint
ETHERSCAN_ENDPOINT = 'https://api.etherscan.io/api'

# Path to the ERC20 ABI
ERC20_ABI_PATH = 'abi/erc20.json'

# True to dump transactions to a json file
DUMP_TRANS = True

## Load environment variables

In [4]:
dotenv_path = Path('.env/uniswap')
load_dotenv(dotenv_path=dotenv_path)
PROVIDER_URL = os.getenv('PROVIDER_URL')
SMART_CONTRACT = os.getenv('SMART_CONTRACT')
NFT_CONTRACT = os.getenv('NFT_CONTRACT')
WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
ETHERSCAN_API_KEY = os.getenv('ETHERSCAN_API_KEY')
SMART_CONTRACT_ABI = os.getenv('SMART_CONTRACT_ABI')

## Calculate the start Block no (optional)

In [5]:
# This is the startig date (DAYS befoe the current date)
d = datetime.today() - timedelta(DAYS)
ts_days_ago = int(d.timestamp())
# We can use this as a starting block
params = {'module':'block', 'action':'getblocknobytime', 'timestamp':{ts_days_ago}, 'closest':'before', 'apikey':{ETHERSCAN_API_KEY}}
# Store the value from the result
start_block = json.loads(requests.get(url=ETHERSCAN_ENDPOINT, params=params).text)['result']
# start_block

## Load the ABIs

In [6]:
# Load the previously downloaded smart contract; unable to download it - Contract source code not verified
with open(SMART_CONTRACT_ABI) as f:
    smart_abi = json.load(f)

# Load the previously downloaded ERC20 ABI
with open(ERC20_ABI_PATH) as f:
    erc20_abi = json.load(f)

# Get ABI of contract; use etherscan API for verifiable smart contracts ABI
params = {'module':'contract', 'action':'getabi', 'address':{NFT_CONTRACT}, 'apikey':{ETHERSCAN_API_KEY}}
response = json.loads(requests.get(url=ETHERSCAN_ENDPOINT, params=params).text)
if response['status'] == '1':
    nft_abi = json.loads(response['result'])
else:
    nft_abi = None

## Create Smart and NFT contracts

In [7]:
# Initialize web3 instance
web3 = Web3(Web3.HTTPProvider(PROVIDER_URL))
smart_contract = web3.eth.contract(address=web3.to_checksum_address(SMART_CONTRACT), abi=smart_abi)
nft_contract = web3.eth.contract(address=web3.to_checksum_address(NFT_CONTRACT), abi=nft_abi)

## Token info

In [8]:
# Token0 info
token0_info = {}
address = smart_contract.functions.token0.call()
# Create an instance of the token0
contract = web3.eth.contract(address=address, abi=erc20_abi)
token0_info['symbol'] = contract.functions.symbol.call()
token0_info['decimals'] = contract.functions.decimals.call()

# Token1 info
token1_info = {}
address = smart_contract.functions.token1.call()
# Create an instance of the token1
contract = web3.eth.contract(address=address, abi=erc20_abi)
token1_info['symbol'] = contract.functions.symbol.call()
token1_info['decimals'] = contract.functions.decimals.call()

## Calcualte Event Signatures

In [9]:
def event_signature(abi:str, type:Contract_Type) -> {str:str}:
    """ Return event singatures for given ABI type
    Parameters:
    abi : str
        ABI contract
    type : Contract_Type
        Either SMART or NFT
        
    Returns:
    dict
        A dictionary signature hex -> event name
    """
    # Dictionary to return (hex -> name)
    signature_to_name = {}
    
    # Loop through abi skipping the first one
    for item in abi[1:]:
        # Only interested in Event types and events in scope
        if ((item['type'] == 'event') and (item['name'] in CONTRACT_EVENTS[type])):
            name = item['name']
            inputs = [param['type'] for param in item['inputs']]
            inputs = ','.join(inputs)
            # Hash event signature
            event_signature_text = f'{name}({inputs})'
            event_signature_hex = web3.to_hex(web3.keccak(text=event_signature_text))
            signature_to_name[event_signature_hex] = name
    return signature_to_name

## Event Signatures for Smart and NFT Contracts

In [10]:
# Dict to store signatures signature -> event name
event_signatures = {}
event_signatures[Contract_Type.SMART] = event_signature(smart_abi, Contract_Type.SMART)
event_signatures[Contract_Type.NFT] = event_signature(nft_abi, Contract_Type.NFT)
event_signatures

{<Contract_Type.SMART: 'smart'>: {'0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c': 'Burn',
  '0x70935338e69775456a85ddef226c395fb668b63fa0115f5f20610b388e6ca9c0': 'Collect',
  '0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde': 'Mint'},
 <Contract_Type.NFT: 'nft'>: {'0x40d0efd1a53d60ecbf40971b9daf7dc90178c3aadc7aab1765632738fa8b8f01': 'Collect',
  '0x26f6a048ee9138f2c0ce266f322cb99228e8d619ae2bff30c67f8dcf9d2377b4': 'DecreaseLiquidity',
  '0x3067048beee31b25b2f1681f88dac838c8bba36af25bfb2b7cf7473a5847e35f': 'IncreaseLiquidity'}}

## Utility method to check an addreess has a contract or not
### Reference: __[How to detect an address is a contract?](https://ethereum.stackexchange.com/questions/28521/how-to-detect-if-an-address-is-a-contract)__

In [11]:
def is_contract_address(address):
    return web3.to_hex(web3.eth.get_code(web3.to_checksum_address(address))) != '0x'

## Utility method to get transactions

In [12]:
def get_transactions(action:str, start_block:int) -> list[str]:
    """ Return a list of transactions singatures for the wallet
    Parameters:
    action : str
        action parameter to the API, txlist, tokennfttx etc.
    start_block: int
        start block to start the search

    Returns:
    list
        A list of transactions for the wallet
    """
    txlist = []
    for page in range(MAX_PAGES):
        params = {'module':'account', 'action':{action}, 'address':{WALLET_ADDRESS}, 'page':{page+1},
                  'offset':{PAGE_SIZE}, 'startblock':{start_block}, 'toBlock':99999999, 'sort':'asc', 'apikey':{ETHERSCAN_API_KEY}}
        txlist_response = json.loads(requests.get(url=ETHERSCAN_ENDPOINT, params=params).text)
        if txlist_response['status'] == '0': break
        txlist = txlist + txlist_response['result']
    return txlist    

## Get a list of Normal and NFT transactions for Wallet

In [13]:
# Normal transactions
trans = get_transactions(action='txlist', start_block=start_block)
# Filter out non contract and error transactions, only interested in hash and timestamp
txlist = [x['hash'] + ':' + x['timeStamp'] for x in trans if is_contract_address(x['to']) and x['isError']=='0']

# NFT transactions
trans = get_transactions(action='tokennfttx', start_block=start_block)
nftlist = [x['hash'] + ':' + x['timeStamp'] for x in trans]

# Merge two lists, remove any duplicates
unique_txlist = list(set(txlist + nftlist))  

## Process Logs

In [14]:
# List of logs
logs = []

# We alawys start with smart contract search
search_type = Contract_Type.SMART
for tx in unique_txlist:
    # Split into hash and timestamp
    (tran_hash, time_stamp) = tx.split(':')
    # Get transaction receipt
    receipt = web3.eth.get_transaction_receipt(tran_hash)
    for log in receipt.logs:
        # Check for smart contract search
        if Contract_Type.SMART == search_type:                
            # Only interested in our smart contract addresses
            if SMART_CONTRACT != log.address: continue
            # Key to search in the signature dict
            key = web3.to_hex(log['topics'][0])
            # Check key is in our interested signatures
            if key in event_signatures[Contract_Type.SMART]:
                # Switch over to NFT search
                search_type = Contract_Type.NFT
        # Check for NFT search
        else:
            # Only interested in our NFT contract addresses
            if NFT_CONTRACT != log.address: continue
            # Key to search in the signature dict
            key = web3.to_hex(log['topics'][0])
            if key in event_signatures[Contract_Type.NFT]:
                # We are interesed in this event
                event_name = event_signatures[Contract_Type.NFT][key]
                # Process logs for this event
                processed_log = nft_contract.events[event_name]().process_log(log)
                token_id = processed_log['args']['tokenId']
                amount0_fees = processed_log['args']['amount0']
                amount1_fees = processed_log['args']['amount1']
                log_index = processed_log['logIndex']
                # Are we collecting from a withdrawl? if so, the this will include the fees. we need to deduct the withdraw from collect
                # to accurately reflect the fees                
                if event_name == 'Collect':
                    # Ensure we have at least one previous log entry
                    if len(logs) > 0:
                        prev_log = logs[-1]
                        # Want to ensure that we only this for when Withdraw/Collect happen within the same transaction
                        if ((prev_log['token_id'] == token_id) and (prev_log['hash'] == tran_hash) and (prev_log['action'] == 'Withdraw')):
                            # Adjust fees
                            amount0_fees = amount0_fees - prev_log['amount0']
                            amount1_fees = amount1_fees - prev_log['amount1']
                
                log = {'token_id':token_id, 'time_stamp':int(time_stamp), 'hash':tran_hash, 'log_index':log_index,
                               'action':NFT_EVENT_TO_ACTION[event_name], 'amount0':amount0_fees, 'amount1':amount1_fees}
                logs.append(log)
                # Switch over to NFT search
                search_type = Contract_Type.SMART

In [15]:
def dump_transactions(data) -> None:
    """ Dumps given data to a json file
    Parameters:
    data : 
        obhect to dump

    Returns:
    None
    """
    out_dir = 'lp_transactions'
    filename = '{path}{sep}{part1}-{part2}.json'.\
        format(path=out_dir, sep=os.sep, part1=WALLET_ADDRESS[:6], part2=SMART_CONTRACT[:6])
    
    # Write to the file
    with open(filename, 'w') as fout:
        json.dump(data , fout)

In [16]:
if DUMP_TRANS:
    dump_transactions(data=logs)
    
token0_col = token0_info['symbol']
token1_col = token1_info['symbol']

# Create a DF from list of logs
df = pd.DataFrame.from_records(data=logs)
df.columns = ['TokenId', 'Timestamp', 'Hash', 'LogIndex', 'Action', token0_col, token1_col]
df = df.sort_values(by=['TokenId','Timestamp', 'LogIndex'])
df['Timestamp'] = df['Timestamp'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x)))
# Apply token decimals
token_divisor = int('1' + ('0' * token0_info['decimals']))
df[token0_col] = df[token0_col].apply(lambda x : x / token_divisor)
token_divisor = int('1' + ('0' * token1_info['decimals']))
df[token1_col] = df[token1_col].apply(lambda x : x / token_divisor)
df.head()

,TokenId,Timestamp,Hash,LogIndex,Action,USDC,WETH
18,903752,2025-01-09 05:27:23,0x1ad2cc022af3e37ce15ae5055e8ee24e4bbf226a3b99...,218,Deposit,6561.830118,2.030305
6,903752,2025-01-10 23:59:11,0xd0b32d113cb235e59b48c4021fcd19ba1cb31c745109...,173,Collect Fee,83.241689,0.026011
40,903752,2025-01-13 07:27:35,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,31,Withdraw,919.460129,3.756619
41,903752,2025-01-13 07:27:35,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,35,Collect Fee,36.381515,0.011590
42,907493,2025-01-13 07:27:35,0xb9f1bc01f7a097c839afa05697f4ea5242334c94c742...,58,Deposit,6366.530436,2.072485


## Utility methods for styling

In [17]:
def style_action(column):    
    deposit_style = 'color: firebrick;'
    default = 'color: forestgreen'

    # must return one string per cell in this column
    return [deposit_style if v == 'Deposit' else default for v in column]

def make_clickable(val):
    url = 'https://etherscan.io/tx/{hash}'.format(hash=val)
    link_name = '{first_part} ... {last_part}'.format(first_part=val[:8], last_part=val[-4:])
    return f'<a target="_blank" href="{url}">{link_name}</a>'

## Apply styles to DF

In [18]:
# Table title
table_caption = 'LP events for the wallet {first_part} ... {last_part}'.\
    format(first_part=WALLET_ADDRESS[:4], last_part=WALLET_ADDRESS[-4:])

# Apply styles
df.style.\
    hide(axis='index').\
    set_caption(table_caption).\
    set_properties(**{'border': '0.1px solid black'}).\
    set_properties(subset=['Action'], **{'text-align': 'left'}).\
    set_table_styles([
        {'selector': 'th.col_heading', 'props': 'text-align: center'},
        {'selector': 'caption', 'props': [('text-align', 'center'), ('font-size', '12pt')]}]).\
    apply(style_action, subset=['Action'], axis=0).\
    format({'Hash': make_clickable, token0_col: '{:.2f}'})

TokenId,Timestamp,Hash,LogIndex,Action,USDC,WETH
903752,2025-01-09 05:27:23,0x1ad2cc ... b917,218,Deposit,6561.83,2.030305
903752,2025-01-10 23:59:11,0xd0b32d ... 8f52,173,Collect Fee,83.24,0.026011
903752,2025-01-13 07:27:35,0xb9f1bc ... e4e3,31,Withdraw,919.46,3.756619
903752,2025-01-13 07:27:35,0xb9f1bc ... e4e3,35,Collect Fee,36.38,0.011590
907493,2025-01-13 07:27:35,0xb9f1bc ... e4e3,58,Deposit,6366.53,2.072485
907493,2025-01-16 08:46:23,0x38e94c ... f1f8,408,Collect Fee,115.14,0.035420
907493,2025-01-16 09:13:11,0x2a00b9 ... 45fa,203,Withdraw,10716.83,0.735192
907493,2025-01-16 09:13:11,0x2a00b9 ... 45fa,207,Collect Fee,0.85,0.000403
909702,2025-01-16 09:13:11,0x2a00b9 ... 45fa,227,Deposit,6206.01,2.095518
909702,2025-01-18 21:06:11,0x2866ea ... bd9b,200,Collect Fee,73.90,0.022255
